In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from tetris_main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao


In [2]:
grid = np.zeros((10,10))
grid_shape = (8, 8)
pieces = pieces_generator(grid_shape)
grid, pieces_coordinates = tetrimino_fitter(pieces, grid_shape)

In [3]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,),
        yaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,),
    ),
)
fig.write_html(f"graphs/test_{grid.size}_size.html")
fig.show()


In [5]:
pop_size = 3
pieces_list = pieces_generator(grid_shape, rotation=False)
# Generate each individual with hill climbing
individuals = [generate_individual(pieces_list, grid_shape) for i in range(pop_size)]

In [6]:
individuals

[Individual <4476.0>, Individual <4504.0>, Individual <5278.0>]

In [5]:
pop = Population(individuals, "max", 2, pieces_list, grid_shape)
gao.GAO(
    pop,
    50,
    tournament_size=4,
    p_crossover=0.05,
    p_mutation=0.1,
    n_mutations=10,
    p_rot_mut=0.1,
    p_adoption=0.05,
)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=600,
        height=600,
        title=f"Fitness: {int(pop.elites[0].fitness)}/{int(np.sum(100*np.ones_like(grid)))} // Pieces used: {len(pieces_coordinates)}/{len(pieces_list)}",
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,),
        yaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,),
    ),
)
fig.show()


In [6]:
px.line(pop.fitness_history)

In [ ]:
# Threshold Acceptance Algorithm

In [7]:
# Hill Climbing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
hill_climb(pop, hardstop=1)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()

In [8]:
# Simulated Annealing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
simulated_annealing(pop)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()